In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from pysat.formula import CNF
import networkx as nx
import numpy as np
import pickle

import os

def _build_graph(cnf_file, output_file):
    cnf = CNF(cnf_file)
    nv = cnf.nv
    clauses = list(filter(lambda x: x, cnf.clauses))
    ind = { k:[] for k in np.concatenate([np.arange(1, nv+1), -np.arange(1, nv+1)]) }
    edges = []
    for i, clause in enumerate(clauses):
        a = clause[0]
        b = clause[1]
        c = clause[2]
        aa = 3 * i + 0
        bb = 3 * i + 1
        cc = 3 * i + 2
        ind[a].append(aa)
        ind[b].append(bb)
        ind[c].append(cc)
        edges.append((aa, bb))
        edges.append((aa, cc))
        edges.append((bb, cc))

    for i in np.arange(1, nv+1):
        for u in ind[i]:
            for v in ind[-i]:
                edges.append((u, v))

    G = nx.from_edgelist(edges)

    # write graph object to output file
    with open(output_file, "wb") as f:
        pickle.dump(G, f, pickle.HIGHEST_PROTOCOL)


# gut = []

# with open('./sat_test_gpickle_files.txt') as f:
#     gut = f.read().splitlines() 

# print(gut)

# print("hi")

# graph_directory = "./SATLIB/"

# datasets = []
# i = 0
# for filename in os.listdir(graph_directory):
#     if filename.endswith(".cnf"):
#         if f"{filename[:-4]}.gpickle" in gut:
#             i = i+1
#             print(i)
#             _build_graph(f"./SATLIB/{filename}", f"../../graphs/satlib/{filename[:-4]}.gpickle")

## Import Graphs

In [3]:
import os
import networkx as nx
import pickle

graph_directory = "../../graphs/satlib"

datasets = []

for filename in os.listdir(graph_directory):
    if filename.endswith(".gpickle"):
        print("3SAT Graph ", os.path.join(graph_directory, filename), "is being imported ...")
        with open(os.path.join(graph_directory, filename), 'rb') as f:
            G = pickle.load(f)
            datasets.append(
            {
                "name": filename[:-8],
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            })

datasets = datasets[0:5] + datasets[7:8] + datasets[10:13] + datasets[14:16] + datasets[17:18] + datasets[37:38] + datasets[39:40] + datasets[19:20] + datasets[21:22]
print(len(datasets))
for dataset in datasets:
    print(dataset["name"])


3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m403_b50_58.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m441_b90_771.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m418_b10_512.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m423_b10_718.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m403_b50_737.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m418_b90_982.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m423_b50_757.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m403_b30_404.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m429_b50_277.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m423_b70_612.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_m441_b10_727.gpickle is being imported ...
3SAT Graph  ../../graphs/satlib/CBS_k3_n100_

In [4]:
from solvers.dNNMIS_GPU_TAU import DNNMIS as V2
from solvers.KaMIS import ReduMIS
from solvers.Quadratic_SDP import Quadratic_SDP
from solvers.Quadratic import Quadratic

solvers = [
    # {
    #     "name": "dNN V2 GPU",
    #     "class": V2,
    #     "params": {
    #         "learning_rate": 0.05,
    #         "selection_criteria": 0.45,
    #         "max_steps": 20000,
    #         "use_cpu": True
    #     },
    # },
    {
        "name": "Quadratic Standard",
        "class": Quadratic_SDP,
        "params": {
            "learning_rate": 0.9,
            "number_of_steps": 50000,
            "gamma": 1500,
            "beta": 1,
            "lr_gamma": 0.1,
            "threshold": 0.0,
        },
    },
    
    #     {
    #     "name": "G775LR0.5S5000",
    #     "class": Quadratic_SDP,
    #     "params": {
    #         "learning_rate": 0.5,
    #         "number_of_steps": 5000,
    #         "gamma": 775,
    #         "batch_size": 1,
    #         "lr_gamma": 0.1,
    #         "threshold": 0.0,
    #     },
    # },
    # {
    #     "name": "dNN",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.8,
    #         "max_steps": 25000,
    #     },
    # },
    #     {
    #     "name": "dNN w/SG5k",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.65,
    #         "max_steps": 80000,
    #         "max_subgraph_steps": 5000,
    #     },
    # },
    # {"name": "ReduMIS", "class": ReduMIS, "params": {"seed": 13}},
    # {"name": "ILP", "class": ILPMIS, "params": {"time_limit": 935}}
]

In [5]:
from copy import deepcopy

def benchmark(datasets, solvers):
    solutions = []

    stage = 0
    stages = len(solvers) * len(datasets)

    for solver in solvers:
        for dataset in datasets:
            solver_instance = solver["class"](dataset["graph"], solver["params"])
            solver_instance.solve()
            solution = {
                "solution_method": solver["name"],
                "dataset_name": dataset["name"],
                "data": deepcopy(solver_instance.solution),
                "time_taken": deepcopy(solver_instance.solution_time),
            }
            solutions.append(solution)
            del solver_instance
            stage += 1
            print(f"Completed {stage} / {stages}")

    return solutions

solutions = benchmark(datasets, solvers)

/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic_SDP.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W_SDP_uv_tensor = torch.tensor(W_SDP_uv, dtype=torch.float32)
/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic_SDP.py:31: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  -(Matrix_X.T @ s_SDP_v_tensor)


using device:  cpu
MIS FOUND WITH SIZE 398, BEST SO FAR IS 398
MIS FOUND WITH SIZE 394, BEST SO FAR IS 398
MIS FOUND WITH SIZE 400, BEST SO FAR IS 400
MIS FOUND WITH SIZE 398, BEST SO FAR IS 400
MIS FOUND WITH SIZE 398, BEST SO FAR IS 400
MIS FOUND WITH SIZE 398, BEST SO FAR IS 400
MIS FOUND WITH SIZE 398, BEST SO FAR IS 400
MIS FOUND WITH SIZE 399, BEST SO FAR IS 400
MIS FOUND WITH SIZE 399, BEST SO FAR IS 400
MIS FOUND WITH SIZE 399, BEST SO FAR IS 400
MIS FOUND WITH SIZE 396, BEST SO FAR IS 400
MIS FOUND WITH SIZE 396, BEST SO FAR IS 400
MIS FOUND WITH SIZE 396, BEST SO FAR IS 400
MIS FOUND WITH SIZE 396, BEST SO FAR IS 400
MIS FOUND WITH SIZE 397, BEST SO FAR IS 400
MIS FOUND WITH SIZE 397, BEST SO FAR IS 400
MIS FOUND WITH SIZE 397, BEST SO FAR IS 400
MIS FOUND WITH SIZE 397, BEST SO FAR IS 400
MIS FOUND WITH SIZE 395, BEST SO FAR IS 400
MIS FOUND WITH SIZE 397, BEST SO FAR IS 400
MIS FOUND WITH SIZE 398, BEST SO FAR IS 400
MIS FOUND WITH SIZE 397, BEST SO FAR IS 400
MIS FOUND WIT

/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic_SDP.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W_SDP_uv_tensor = torch.tensor(W_SDP_uv, dtype=torch.float32)


using device:  cpu
MIS FOUND WITH SIZE 427, BEST SO FAR IS 427
MIS FOUND WITH SIZE 426, BEST SO FAR IS 427
MIS FOUND WITH SIZE 427, BEST SO FAR IS 427
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 428, BEST SO FAR IS 428
MIS FOUND WITH SIZE 427, BEST SO FAR IS 428
MIS FOUND WITH SIZE 427, BEST SO FAR IS 428
MIS FOUND WITH SIZE 425, BEST SO FAR IS 428
MIS FOUND WITH SIZE 429, BEST SO FAR IS 429
MIS FOUND WITH SIZE 429, BEST SO FAR IS 429
MIS FOUND WITH SIZE 431, BEST SO FAR IS 431
MIS FOUND WITH SIZE 432, BEST SO FAR IS 432
MIS FOUND WITH SIZE 430, BEST SO FAR IS 432
MIS FOUND WITH SIZE 433, BEST SO FAR IS 433
MIS FOUND WITH SIZE 433, BEST SO FAR IS 433
MIS FOUND WITH SIZE 434, BEST SO FAR IS 434
MIS FOUND WITH SIZE 433, BEST SO FAR IS 434
MIS FOUND WIT

In [ ]:
import pandas
import matplotlib.pyplot as plt

dataset_index = {k: v for v, k in enumerate([dataset["name"] for dataset in datasets])}
datasets_solutions = [[] for i in range(len(datasets))]

table_data = []

for solution in solutions:
    dsi = dataset_index[solution["dataset_name"]]
    datasets_solutions[dsi].append(solution)

i = 0
for dataset_solutions in datasets_solutions:
    # IS CHECK
    is_check = []
    for solution in dataset_solutions:
        IS_set = solution["data"]["graph_mask"]
        subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(
            IS_set
        )
        # if len(subgraph.edges) > 0:
        #     plt.figure(i)
        #     plt.title(subgraph.edges)
        #     i += 1
        #     nx.draw(datasets[dataset_index[solution["dataset_name"]]]["graph"], with_labels=True, node_color=IS_set)
        #     is_check.append(False)
        #     print(
        #         f"Non IS found using {solution['solution_method']} on {solution['dataset_name']}"
        #     )
        # else:
        is_check.append(True)

    table_row = [dataset_solutions[0]['dataset_name']]

    table_row.extend([solution["data"]["size"] for solution in dataset_solutions])
    table_row.extend([solution["time_taken"] for solution in dataset_solutions])
    table_row.extend(is_check)

    table_data.append(table_row)

table_headers = ["Dataset Name"]

table_headers.extend([solver["name"] + " Solution Size" for solver in solvers])
table_headers.extend([solver["name"] + " Solution Time" for solver in solvers])
table_headers.extend([solver["name"] + " Solution IS" for solver in solvers])

table = pandas.DataFrame(table_data, columns=table_headers)
table
table.to_csv("results.csv")